In [ ]:
import os
import os.path as osp
import pandas as pd
from pyarrow import parquet as pq
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Trying to get some sense out of the metadata

Opening one random target file in the folder. The metadata is very messy:

Each table is stored in a Parquet file which can be processed by different data processing libraries like Pandas, Spark, and Pyarrow. 

Each Parquet file consists of the table itself and its metadata, consisting of:
    
    URL to the original CSV file,
    License of the associated repository,
    Table ID,
    Table dimensions,
    Data types inferred with Pandas,
    Column annotations from different annotation methods and ontologies,
    Table topic annotation derived from column annotations.


We'll have to take the annotations into consideration when we try to use these tables. 

Overall, I conducted a random, cursory search on the tables, opening a bunch of 
them out of curiosity. Tables are very loosely connected to each other by archive, 
the content is unformatted, contains a bunch of mixed data (numerical columns, 
categorical columns, text columns, IDs, lists...). The metadata is similarly 
messy. 

In [ ]:
table.schema

AttributeError: 'DataFrame' object has no attribute 'schema'

In [ ]:
pwd

'/home/soda/rcappuzz/work/study-gittables'

In [ ]:
root_dir = "data/zenodo/tables/"
folder_name = "allegro_con_spirito_tables_licensed"
tgt_folder = osp.join(root_dir, folder_name)


In [ ]:
table = pq.read_table(osp.join(tgt_folder, 'incantesimo.parquet'))
md = table.schema.metadata[b'gittables']

b'{"license": "GNU General Public License v3.0", "csv_url": "https://github.com/ICosmos34/Basi-di-Dati/blob/ab0792494c9db586c2634c35a644afeaada2f761/Dati/incantesimo.csv", "dtypes": {"1": "int64", "Abilit\\u00e0 in Prestito": "object", "1.1": "int64", "Si effettua una prova di Abilit\\u00e0 usando i gradi di un altro": "object"}, "number_rows": 1772, "number_columns": 4, "dtypes_percentages": {"int64": 0.5, "object": 0.5}, "dbpedia_syntactic_column_types": {}, "schema_syntactic_column_types": {}, "dbpedia_semantic_column_types": {"Si effettua una prova di Abilit\\u00e0 usando i gradi di un altro": {"cleaned_label": "e teatr id", "description": null, "domain": ["http://dbpedia.org/ontology/Film"], "id": "http://dbpedia.org/ontology/eTeatrId", "range": ["http://www.w3.org/2001/XMLSchema#string"], "superproperty": null}}, "dbpedia_semantic_similarities": {"Si effettua una prova di Abilit\\u00e0 usando i gradi di un altro": 0.63}, "schema_semantic_column_types": {"Si effettua una prova di 